In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [12]:
# Гиперпараметры
num_epochs = 5
learning_rate = 0.001
batch_size = 128
num_classes = 10

In [13]:
# Подготовка данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# Определение остаточного блока
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Определение свёрточной сети с остаточными блоками
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [15]:
# Проверка наличия GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Инициализация модели, функции потерь и оптимизатора
model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes).to(device) # GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device) # GPU

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if ((i+1) % 100 == 0):
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Сохранение модели
model_path = 'resnet_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')
model.to(device) # GPU

# Тестирование модели
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) # GPU

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Using device: cpu
Epoch [1/5], Step [100/391], Loss: 1.3085
Epoch [1/5], Step [200/391], Loss: 1.2092
Epoch [1/5], Step [300/391], Loss: 1.0090
Epoch [1/5], Step [391/391], Loss: 0.8913
Epoch [2/5], Step [100/391], Loss: 0.9036
Epoch [2/5], Step [200/391], Loss: 0.8912
Epoch [2/5], Step [300/391], Loss: 0.7201
Epoch [2/5], Step [391/391], Loss: 0.5549
Epoch [3/5], Step [100/391], Loss: 0.5348
Epoch [3/5], Step [200/391], Loss: 0.5226
Epoch [3/5], Step [300/391], Loss: 0.4684
Epoch [3/5], Step [391/391], Loss: 0.4706
Epoch [4/5], Step [100/391], Loss: 0.4760
Epoch [4/5], Step [200/391], Loss: 0.5874
Epoch [4/5], Step [300/391], Loss: 0.3173
Epoch [4/5], Step [391/391], Loss: 0.5565
Epoch [5/5], Step [100/391], Loss: 0.3831
Epoch [5/5], Step [200/391], Loss: 0.4551
Epoch [5/5], Step [300/391], Loss: 0.2744
Epoch [5/5], Step [391/391], Loss: 0.2929
Model saved to resnet_model.pth
Accuracy of the model on the test images: 80.02 %


In [17]:
# Загрузка модели
model_path = 'resnet_model.pth'
model.load_state_dict(torch.load(model_path))
model.to(device) # GPU

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) # GPU

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Accuracy of the model on the test images: 80.02 %
